In [59]:
import tensorflow as tf
import time

In [60]:
# szlaki do danych
train_dir = r"C:\MyComputer\uni 2 uj\SI\FruitRecognition\Fruit-Images-Dataset\Training"
test_dir  = r"C:\MyComputer\uni 2 uj\SI\FruitRecognition\Fruit-Images-Dataset\Test"

In [61]:
# poczatkowe naladowania
SEED = 1
BATCH_SIZE = 32

IMG_SIZE = (100, 100)

AUTOTUNE = tf.data.AUTOTUNE # naladowanie parametrow

In [62]:
# instalowanie datasetow
def make_datasets(img_size):

    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        image_size=img_size,
        batch_size=BATCH_SIZE,
        seed=SEED,
        shuffle=True,
        validation_split=0.2,
        subset="training",
        label_mode="int"
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        image_size=img_size,
        batch_size=BATCH_SIZE,
        seed=SEED,
        shuffle=False,
        validation_split=0.2,
        subset="validation",
        label_mode="int"
    )

    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        image_size=img_size,
        batch_size=BATCH_SIZE,
        seed=SEED,
        shuffle=False,
        label_mode="int"
    )

    class_names = train_ds.class_names
    num_classes = len(class_names)
    print("Classes:", num_classes)

    # dla szybszego pipelina
    train_ds = train_ds.prefetch(AUTOTUNE)
    test_ds = test_ds.prefetch(AUTOTUNE)

    return train_ds, test_ds, val_ds, class_names, num_classes

In [63]:
# prosta CNN
def build_simple_cnn(num_classes, img_size):
    # augmentation
    aug = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"), # lustrowe
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])

    # CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Input((img_size[0], img_size[1], 3)), # 3 -- RGB
        aug,
        tf.keras.layers.Rescaling(1./255),

        tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),

        tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3), # wylaczenie 30% neuronow, zapobiega przeuczeniu
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

    return model

In [64]:
print("\nProsta CNN")
start = time.time()

train_ds, test_ds, val_ds, class_names, num_classes = make_datasets(IMG_SIZE)
model1 = build_simple_cnn(num_classes, IMG_SIZE)
model1.summary()

hist1 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

test_loss, test_acc = model1.evaluate(test_ds, verbose=0)
print(f"Dla prostej CNN test accuracy: {test_acc*100:.2f}%")

end = time.time()
print(f"\nCzas dla prostej CNN: {end-start:.2f} sekund")


Prosta CNN
Found 67692 files belonging to 131 classes.
Using 54154 files for training.
Found 67692 files belonging to 131 classes.
Using 13538 files for validation.
Found 22688 files belonging to 131 classes.
Classes: 131


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_7 (Rescaling)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 50, 50, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 80000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 80000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 131)            │    10,480,131 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,518,019 (40.12 MB)

 Trainable params: 10,518,019 (40.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 480s 282ms/step - accuracy: 0.8461 - loss: 0.5587 - val_accuracy: 0.9747 - val_loss: 0.0789
Epoch 2/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 414s 245ms/step - accuracy: 0.9716 - loss: 0.0942 - val_accuracy: 0.9940 - val_loss: 0.0184
Epoch 3/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 398s 235ms/step - accuracy: 0.9804 - loss: 0.0638 - val_accuracy: 0.9778 - val_loss: 0.0649
Epoch 4/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 400s 236ms/step - accuracy: 0.9815 - loss: 0.0591 - val_accuracy: 0.9955 - val_loss: 0.0176
Epoch 5/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 502s 297ms/step - accuracy: 0.9872 - loss: 0.0439 - val_accuracy: 0.9970 - val_loss: 0.0093
Dla prostej CNN test accuracy: 96.38%

Czas dla prostej CNN: 2237.38 sekund


In [65]:
# transfer learning
def build_transfer_model(num_classes, img_size):
    # EfficientNetB0 -- CNN
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet", # bierzemy z juz edukowanej modeli
        input_shape=(img_size[0], img_size[1], 3)
    )

    base.trainable = False  # weights sie nie zmieniaja

    model = tf.keras.Sequential([
        tf.keras.layers.Input((img_size[0], img_size[1], 3)),
        tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input), # wazna zmiana
        base, # malunek do features map
        tf.keras.layers.GlobalAveragePooling2D(), # jeden wektor z features map
        tf.keras.layers.Dense(num_classes, activation="softmax") # probability
    ])

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model, base

In [66]:
print("\nTransfer learning + fine tuning")
start = time.time()

train_ds, test_ds, val_ds, class_names, num_classes = make_datasets(IMG_SIZE)

model2, base_model = build_transfer_model(num_classes, IMG_SIZE)
model2.summary()

# 1) transfer learning
model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# 2) fine tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:  # tylko 30 ostatnich trenuja sie
    layer.trainable = False

model2.compile(
    optimizer=tf.keras.optimizers.Adam(0.00001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model2.fit(train_ds, validation_data=val_ds, epochs=5)

test_loss, test_acc = model2.evaluate(test_ds, verbose=0)
print(f"Po fine tuningu test accuracy: {test_acc*100:.2f}%")

end = time.time()
print(f"\nCzas dla transfer learningu i fine tuningu: {end-start:.2f} sekund")


Transfer learning + fine tuning
Found 67692 files belonging to 131 classes.
Using 54154 files for training.
Found 67692 files belonging to 131 classes.
Using 13538 files for validation.
Found 22688 files belonging to 131 classes.
Classes: 131



Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 131)            │       167,811 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,217,382 (16.09 MB)

 Trainable params: 167,811 (655.51 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 351s 200ms/step - accuracy: 0.9425 - loss: 0.4143 - val_accuracy: 0.9992 - val_loss: 0.0488
Epoch 2/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 365s 215ms/step - accuracy: 0.9975 - loss: 0.0376 - val_accuracy: 0.9996 - val_loss: 0.0195
Epoch 3/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 476s 281ms/step - accuracy: 0.9988 - loss: 0.0168 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 4/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 447s 264ms/step - accuracy: 0.9995 - loss: 0.0087 - val_accuracy: 0.9999 - val_loss: 0.0040
Epoch 5/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 379s 224ms/step - accuracy: 0.9994 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 1/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 539s 309ms/step - accuracy: 0.9760 - loss: 0.1085 - val_accuracy: 0.9999 - val_loss: 0.0064
Epoch 2/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 508s 300ms/step - accuracy: 0.9954 - loss: 0.0261 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 3/5
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 493s 291ms/step - accuracy: 